In [1]:
import sympy as sp
thp, thd, tp, td, lp, ld, alp, r, s = sp.symbols(r'\theta_p \theta_d t_p t_d l_p l_d \alpha r s')


In [2]:
kda = td / (ld * r)
kd1 = alp * kda

In [3]:
kd = 2 * (kda - kd1) * s / ld + kd1
display(sp.simplify(kd))

t_d*(\alpha*l_d - 2*s*(\alpha - 1))/(l_d**2*r)

In [4]:
thdint = sp.integrate(kd, (s, lp, ld))
display(sp.simplify(thdint))

t_d*(-\alpha*l_d*l_p + l_d**2 + l_p**2*(\alpha - 1))/(l_d**2*r)

In [5]:
thdeqn = thdint - thd
# tdsol = sp.solve(thdeqn, td)[0]
# display(tdsol)

In [6]:
kpa = tp / (lp * r)
kp1 = alp * kpa
kp = 2 * (kpa - kp1) * s / lp + kp1

kptot = kd/2 + kp
display(sp.simplify(kptot))

(\alpha*l_d**2*l_p*t_p + \alpha*l_d*l_p**2*t_d/2 - 2*l_d**2*s*t_p*(\alpha - 1) - l_p**2*s*t_d*(\alpha - 1))/(l_d**2*l_p**2*r)

In [7]:
kpint = sp.integrate(kptot, (s, 0, lp))
display(sp.simplify(kpint))

thpeqn = kpint - thp
# tpsol = sp.solve(thpeqn, tp)[0]
# display(tpsol)

## to be cont'd

(\alpha*l_d*l_p*t_d - \alpha*l_p**2*t_d + 2*l_d**2*t_p + l_p**2*t_d)/(2*l_d**2*r)

In [8]:
# tp_withtd = tpsol.subs(td, tdsol)
# tp_withtd_simple = sp.simplify(tp_withtd)
# display(tp_withtd_simple)
# tpsimple_frac = sp.fraction(tp_withtd_simple)
# display(tpsimple_frac[0].simplify())
# display(tpsimple_frac[1].simplify())


In [9]:
tdlsol_new = sp.solve((thdeqn, thpeqn),('t_p','t_d'))
display(sp.simplify(tdlsol_new[tp]))
display(sp.simplify(tdlsol_new[td]))

r*(\alpha*\theta_d*l_d*l_p - \alpha*\theta_d*l_p**2 + 2*\alpha*\theta_p*l_d*l_p - 2*\alpha*\theta_p*l_p**2 + \theta_d*l_p**2 - 2*\theta_p*l_d**2 + 2*\theta_p*l_p**2)/(2*(\alpha*l_d*l_p - \alpha*l_p**2 - l_d**2 + l_p**2))

-\theta_d*l_d**2*r/(\alpha*l_d*l_p - \alpha*l_p**2 - l_d**2 + l_p**2)